In [77]:
import os
import io
import re
import numpy as np
import pandas as pd
import platform
from PIL import ImageFont, ImageDraw, Image
import matplotlib.pyplot as plt

import cv2
from google.cloud import vision

In [78]:
# OCR API 불러오기
# \ => /로 변환해야 파일 인식 가능(안바꾸면 유니코드 에러 발생함)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users\Hong Ji Young\Desktop/Health-C/health-c-377913-17227fcd7bd5.json'
 
client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [116]:
# 이미지 불러오기
path = 'img2.jpg'
with io.open(path, 'rb') as img_file:
    content = img_file.read()

In [117]:
# 이미지 OCR, 글자 인식(30초 정도 걸림)
image = vision.Image(content=content)
 
response = client.text_detection(image=image)
texts = response.text_annotations

In [118]:
# 읽어온 글자 중 재료명만 골라내기 위한 작업<1>
#     특수문자 제거, 순수한글 & 숫사 & 영어만 가져와서 리스트로 생성
lst = []
special_char = ''
for text in texts[0].description.split(','):
    x = [re.sub('[^A-Za-z0-9가-힣]', '', t) for t in text.split()]
    for j in x:
        lst.append(re.sub('[^A-Za-z0-9가-힣]', '', j))

## 엑셀 파일 읽어오기

In [101]:
# xlsx파일 dataframe으로 생성
materials = pd.read_excel('materials_all.xlsx')

In [102]:
materials.head()

,Unnamed: 0,원재료,대표원재료명,원재료이명
0,0,Abiu열매,Abiu열매,NaN
1,1,Agathi꽃,Agathi꽃,NaN
2,2,Agathi잎,Agathi잎,NaN
3,3,Alder Buckthorn,Alder Buckthorn,NaN
4,4,Algarrobo Blanco,Algarrobo Blanco,NaN


In [119]:
food_materials = pd.DataFrame({'원재료' : list(set(lst))})
result = pd.merge(materials, food_materials
                   , how="inner", right_on="원재료", left_on="대표원재료명")

In [120]:
list(result["대표원재료명"])

['대두',
 '땅콩',
 '밀',
 '밀가루',
 '코코아매스',
 '맥아엑기스',
 '돼지고기',
 '우유',
 '유당',
 '알긴산프로필렌글리콜',
 '효모',
 '단백질',
 '폴리프로필렌',
 '설탕',
 '쇼트닝',
 '초콜릿',
 '가공버터',
 '가공유',
 '전지분골드',
 '준초콜릿',
 '효소제',
 '나트륨']

In [89]:
# 사용하지 않는 코드(참고만)
common_materials = pd.DataFrame()
for txt in lst:
    print(materials[materials['원재료'].str.contains(txt)])
#     print(materials.query('원재료.str.contains(@txt)'), engine="python")

       Unnamed: 0                                               원재료
14697       14697       폴리알킬렌(에틸렌-프로필렌 포함)글리콜부톡시모노에테르(평균분자량 3,300) 
14705       14705                  옥시란과 메틸옥시란의 블록공중합체(평균분자량 1,900) 
14706       14706                옥시란과 메틸옥시란의 블록공중합체(최소평균분자량 2,000) 
14707       14707  옥시란과 메틸옥시란의 블록공중합체 (메틸옥시란이 27～31몰, 평균분자량 2,000) 
14734       14734      폴리알킬렌(에틸렌-프로필렌 포함)글리콜부톡시모노에테르 (최소분자량 2,400) 
...           ...                                               ...
27260       27260                                       COLOUR 100 
27967       27967               POLYETHYLENE GLYCOL (400) DIOLEATE 
27968       27968               POLYETHYLENE GLYCOL (600) DIOLEATE 
27973       27973                   POLYOXYETHYLENE (600) DIOLEATE 
27976       27976            POLYOXYETHYLENE (600) MONORICINOLEATE 

[296 rows x 2 columns]
